Установить требуемую версию torch

In [ ]:
%pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
print(torch.__version__)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 708.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu121
    Uninstalling torchaudio-2.5.1+cu121:
      Successfully uninstalled torchaudio-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

Монтировать диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Копировать файлы из гугл диска в сессионное хранилище

In [ ]:
!cp -r '/content/drive/MyDrive/EasyOCR-master' '/content/'
!ls

drive  EasyOCR-master  sample_data


Распаковать датасет

In [ ]:
!unzip '/content/EasyOCR-master/trainer/all_data/Dataset.zip' -d '/content/EasyOCR-master/trainer/all_data'

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E301OB74_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E302PC35_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E303ET10_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E303HH82_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E304EX45_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E307AX24_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E307EE161_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E307HK777_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E308HH198_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E313HP29_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E316XT198_0.png  
  inflating: /content/EasyOCR-master/trainer/all_data/valid/E316YH178_0.png  
  i

In [ ]:
import os
os.getcwd()
%cd /content/EasyOCR-master/trainer

/content/EasyOCR-master/trainer


In [ ]:
!pip install -r ../requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 38.8 MB/s eta 0:00:00


In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [ ]:
#Запускаем обучение
opt = get_config("config_files/custom_data_train.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: train
all_data/train
sub-directory:	/train	 num samples: 54894
num total samples of train: 54894 x 1.0 (total_data_usage_ratio) = 54894
num samples of train per batch: 64 x 1.0 (batch_ratio) = 64


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    all_data/valid	 dataset: /
all_data/valid/
sub-directory:	/.	 num samples: 7735
--------------------------------------------------------------------------------
No Transformation module specified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model input parameters 64 600 20 1 256 256 208 68 None VGG BiLSTM CTC
loading pretrained model from saved_models/cyrillic_g2.pth
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
        (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
        (10): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
        (11): Conv2d(128, 256, ker

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!cp -r '/content/EasyOCR-master/trainer/saved_models/ru_numbers_eq_data' '/content/drive/MyDrive/EasyOCR-master/trainer/saved_models'